## 1. load Packages

In [1]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
import scvelo as scv
from matplotlib import rcParams

## 2. params

In [2]:
os.chdir("D:/Projects/CASH-Human/")
dir_for_NK_ldata = "./data/loom/ldata_sub/NK_ldata.h5ad"

## setup
scv.settings.verbosity = 0
sc.settings.verbosity = 0
### fig
sc.settings.set_figure_params(dpi = 100, dpi_save = 300, fontsize = 14, frameon = True,figsize = (8,8))
scv.settings.set_figure_params(dpi = 100, dpi_save = 300, fontsize = 14, frameon = None, figsize = (8,8)) 

## 3. load data

In [ ]:
## load data
ldata = scv.read(os.path.join(dir_for_NK_ldata))
print("ldata:\n", ldata)
print("ldata_clusters:\n", pd.unique(ldata.obs["sub_celltype"]))

In [ ]:
cluster_group = ["c28_NK_CD160", "c29_NK_NFKBIA", "c30_NK_DNAJB1", "c31_NK_CCL4"]
# ldata = ldata[ldata.obs["sub_celltype"].isin(cluster_group)]
ldata = ldata[ldata.obs["sub_celltype"].isin(cluster_group)]
print("ldata_sub:\n", ldata)
print("ldata_sub_clusters:\n", pd.unique(ldata.obs["sub_celltype"]))

In [5]:
## preprocess
sc.pp.pca(ldata)
sc.pp.neighbors(ldata, n_pcs = 50, n_neighbors = 30)

In [ ]:
## setup
root_cells = ldata.obs_names[ldata.obs["sub_celltype"] == "c29_NK_NFKBIA"]
root_id = ldata.obs_names.get_indexer([root_cells[0]])[0]
ldata.uns["root"] = root_id
sc.tl.dpt(ldata)

scv.tl.recover_dynamics(ldata, n_jobs=14)
scv.tl.latent_time(ldata, root_key='root')

In [ ]:
# sc.pl.umap(ldata, color = "sub_celltype", frameon = False, legend_loc = "right margin")

## compute
scv.pp.filter_and_normalize(ldata)
scv.pp.moments(ldata)

# scv.tl.velocity(ldata, group_by = 'sub_celltype')

# scv.tl.velocity_graph(ldata)

## plot
scv.pl.velocity_embedding_stream(ldata, 
                                 basis = "X_umap", 
                                 color = "sub_celltype",
                                 title = "",
                                 ## legend
                                 legend_fontsize=12,
                                 legend_loc = "lower center",
                                 ## save
                                 save = "./fig/Fig03.NK/scVelocity_c29-c31_NK.pdf",
                                 density = 1.5, alpha=0.4,linewidth=0.8)